In [ ]:
import numpy as np
from scipy.signal import convolve2d
import tensorflow as tf
import keras
from keras import layers
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from random import randint, random
import sys
import gc
import time

In [ ]:
def life_step(X):
    """Game of life step using scipy tools"""
    X_pad = np.zeros((X.shape[0], X.shape[1]+2, X.shape[2]+2))
    N = np.zeros((X.shape[0], X.shape[1]+2, X.shape[2]+2))
    X_pad[:,1:-1,1:-1] += X
    
    X_pad[:, 0, 1:-1] = X[:, -1, :]
    X_pad[:, -1, 1:-1] = X[:, 0, :]
    
    X_pad[:, 1:-1, 0] = X[:, :, -1]
    X_pad[:, 1:-1, -1] = X[:, :, 0]
    
    X_pad[:, 0, 0] = X[:, -1, -1]
    X_pad[:, 0, -1] = X[:, -1, 0]
    X_pad[:, -1, 0] = X[:, 0, -1]
    X_pad[:, -1, -1] = X[:, 0, 0]
    
    N[:, 1:, 1:] += X_pad[:,:-1,:-1]
    N[:, 1:, :] += X_pad[:,:-1,:]
    N[:, 1:, :-1] += X_pad[:,:-1,1:]

    N[:, :, 1:] += X_pad[:,:,:-1]
    N[:, :, :-1] += X_pad[:,:,1:]

    N[:, :-1, 1:] += X_pad[:,1:,:-1]
    N[:, :-1, :] += X_pad[:,1:,:]
    N[:, :-1, :-1] += X_pad[:,1:,1:]
    
    N = N[:,1:-1,1:-1]

    return np.logical_or(N == 3, np.logical_and(X, N==2)).astype(np.uint8)


In [ ]:
test = pd.read_csv("../input/conways-reverse-game-of-life-2020/test.csv", index_col='id').values

test_delta = test[:,0]

X_test = test[:, 1:626].reshape((-1, 25, 25))

print(test_delta.shape, X_test.shape)

In [ ]:
# Naïve aproaches

# Blank submission
blank_score = X_test.mean(axis=(-2, -1))

# Stop submission
stop_scores = np.zeros_like(blank_score)
current_step = np.copy(X_test)
for i in range(5):
    current_step = life_step(current_step)
    stop_scores[test_delta==i+1] = 1 - (current_step[test_delta==i+1] == X_test[test_delta==i+1]).mean(axis=(-2, -1))

print(blank_score.mean())
print(stop_scores.mean())

In [ ]:
naive_submission = np.zeros_like(X_test)
stop_best_mask = stop_scores < blank_score
naive_submission[stop_best_mask] = X_test[stop_best_mask]
naive_scores = np.copy(blank_score)
naive_scores[stop_best_mask] = stop_scores[stop_best_mask]
    
print(naive_scores.mean())

In [ ]:
game_dim = 25
batch_size = 128

INITIAL_CONV = 64
RES_LAYERS = 5

In [ ]:
class ResnetIdentityBlock(keras.Model):
  def __init__(self, kernel_size, filters):
    super(ResnetIdentityBlock, self).__init__(name='')
    filters1, filters2, filters3 = filters

def get_model(input_shape):
  t_input = keras.layers.Input(input_shape)
  x = keras.layers.Conv2D(INITIAL_CONV, 3, padding='same')(t_input)
  x = keras.layers.BatchNormalization()(x)
  x = keras.activations.relu(x)
  for _ in range(RES_LAYERS):
    x_input = x
    x = keras.layers.Conv2D(INITIAL_CONV, 3, padding='same')(x)
    x = keras.layers.BatchNormalization()(x)
    x = tf.nn.relu(x)

    x = keras.layers.Conv2D(INITIAL_CONV, 3, padding='same')(x)
    x = keras.layers.BatchNormalization()(x)

    x += x_input
    x = keras.layers.Conv2D(1, 3, padding='same')(x)
    x = keras.activations.sigmoid(x)

  return keras.models.Model(inputs=t_input, outputs=x)

def build_model():
  return get_model((game_dim, game_dim, 7))

In [ ]:
model = build_model()

model.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=0.00001, verbose=1)

file_path = 'best_model.hdf5'

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=file_path, monitor='val_loss',
                                                    save_best_only=True)

callbacks = [reduce_lr, model_checkpoint]
#callbacks = [model_checkpoint]

model.summary()

In [ ]:
model.load_weights('../input/iglolpartialmodel/best_model.hdf5')

In [ ]:
p0 = 0.3058699275697248
max_delta = 5
def get_padded_version_n(X):
    X_pad = np.zeros((X.shape[0], X.shape[-2] + 2, X.shape[-1] + 2), dtype=X.dtype)
    X_pad[:, 1:-1,1:-1] += X
    
    X_pad[:, 0, 1:-1] = X[:, -1, :]
    X_pad[:, -1, 1:-1] = X[:, 0, :]
    
    X_pad[:, 1:-1, 0] = X[:, :, -1]
    X_pad[:, 1:-1, -1] = X[:, :, 0]
    
    X_pad[:, 0, 0] = X[:, -1, -1]
    X_pad[:, 0, -1] = X[:, -1, 0]
    X_pad[:, -1, 0] = X[:, 0, -1]
    X_pad[:, -1, -1] = X[:, 0, 0]
    
    return X_pad


def solve(F, d):
    sF = F
    P = p0 * np.ones_like(F)
    
    F_pad = get_padded_version_n(F)
    
    N = np.zeros(F_pad.shape)
    
    N[:, 1:, 1:] += F_pad[:,:-1,:-1]
    N[:, 1:, :] += F_pad[:,:-1,:]
    N[:, 1:, :-1] += F_pad[:,:-1,1:]

    N[:, :, 1:] += F_pad[:,:,:-1]
    N[:, :, :-1] += F_pad[:,:,1:]

    N[:, :-1, 1:] += F_pad[:,1:,:-1]
    N[:, :-1, :] += F_pad[:,1:,:]
    N[:, :-1, :-1] += F_pad[:,1:,1:]

    N = N[:,1:-1,1:-1]
    
    P[np.logical_and(N==0, F==0)] = 0
    
    delta_indic = np.repeat(np.eye(max_delta)[d-1], F.shape[-2] * F.shape[-1], axis=0).reshape((F.shape[0], F.shape[-2], F.shape[-1], -1))
    
    print("Starting from", np.logical_and(N==0, F==0).sum(axis=(-1, -2)) / (F.shape[-2] * F.shape[-1]))
    
    I = np.zeros_like(F)
    global_mask = np.ones(F.shape[0]).astype(np.bool_)
    
    while len(F):
        ts = time.time()
        X = np.concatenate((np.expand_dims(F, axis=-1), np.expand_dims(P, axis=-1), delta_indic), axis=-1)
        y = model.predict(X)[:, :, :, 0]
        del X
        p_0_mask = P==0.0
        p_1_mask = P==1.0
        y[p_0_mask] = 0.0
        y[p_1_mask] = 1.0
        H = abs(y - 0.5)
        H[p_0_mask] = 0.0
        H[p_1_mask] = 0.0
        change_mask = H == np.repeat(H.max(axis=(-1, -2)), F.shape[-2] * F.shape[-1]).reshape(H.shape)
        nP = y
        nP[change_mask] = (np.round(nP[change_mask]) > 0.5).astype(np.float32)
        qsure = ((nP==0.0).sum(axis=(-1, -2)) + (nP==1.0).sum(axis=(-1, -2))) / (F.shape[-2] * F.shape[-1])
        local_continue_mask = qsure < 1.0
        local_stop_mask = np.logical_not(local_continue_mask)
        
        if local_stop_mask.sum() > 0:
            eq = (life_step(nP[local_stop_mask]) == F[local_stop_mask])
            I[np.arange(len(I))[global_mask][local_stop_mask]] = nP[local_stop_mask]
            d[np.arange(len(d))[global_mask][local_stop_mask]] -= 1
            F[local_stop_mask] = nP[local_stop_mask]
            
            nF = F[local_stop_mask]
            n_p = p0 * np.ones_like(nF)
            nF_pad = get_padded_version_n(nF)

            nN = np.zeros_like(nF_pad)

            nN[:, 1:, 1:] += nF_pad[:,:-1,:-1]
            nN[:, 1:, :] += nF_pad[:,:-1,:]
            nN[:, 1:, :-1] += nF_pad[:,:-1,1:]

            nN[:, :, 1:] += nF_pad[:,:,:-1]
            nN[:, :, :-1] += nF_pad[:,:,1:]

            nN[:, :-1, 1:] += nF_pad[:,1:,:-1]
            nN[:, :-1, :] += nF_pad[:,1:,:]
            nN[:, :-1, :-1] += nF_pad[:,1:,1:]

            nN = nN[:,1:-1,1:-1]
            
            n_p[np.logical_and(nN==0, nF==0)] = 0.0
            nP[local_stop_mask] = n_p
            
            delta_indic[local_stop_mask] = np.repeat(np.eye(max_delta)[d[global_mask][local_stop_mask]-1], F.shape[-2] * F.shape[-1], axis=0).reshape((local_stop_mask.sum(), F.shape[-2], F.shape[-1], -1))
            
            local_continue_mask[local_stop_mask] = d[global_mask][local_stop_mask] > 0
            F = F[local_continue_mask]
            nP = nP[local_continue_mask]
            delta_indic = delta_indic[local_continue_mask]
            """
            print("Done for {}, steps {}, continuing with {} items".format(
                np.arange(len(I))[global_mask][local_stop_mask],
                d[global_mask][local_stop_mask],
                local_continue_mask.sum()
            ))
            """
            global_mask[global_mask] = local_continue_mask
            
            del eq
            del nN
            del nF
            del nF_pad
        P = nP
        print("{} items left scores: min={:.3f}, max={:.3f}, mean={:.3f} {:.3f}s".format(
            local_continue_mask.sum(),
            qsure.min(),
            qsure.max(),
            qsure.mean(),
            time.time() - ts
        ))
        del y
        del H
        del change_mask
        del qsure
        del local_continue_mask
        del local_stop_mask
        gc.collect()

    return I

In [ ]:
def get_optimized_solution_and_score(y, delta, labels, naive_submission, naive_scores):
    print("Scores from prediction")
    res = np.zeros_like(y)
    scores = []
    current_step = np.copy(y)
    for i in range(5):
        current_step = life_step(current_step)
        scores.append(current_step[delta==i+1] == labels[delta==i+1])
        print("Actual score for delta={}: {}".format(i+1, scores[-1].mean()))
        res[delta == i+1] = current_step[delta == i+1]
    print("Actual LB score = ", 1 - (res == labels).mean())
    y_scores = 1 - (labels == res).mean(axis=(-2, -1))
    naive_mask = naive_scores < y_scores
    y_final = np.copy(y)
    y_final[naive_mask] = naive_submission[naive_mask]
    f_score = np.copy(y_scores)
    f_score[naive_mask] = naive_scores[naive_mask]
    print("LB score estimation = ", f_score.mean())
    return y_final, f_score.mean()

In [ ]:
y = solve(X_test, np.copy(test_delta))

In [ ]:
y_final, score = get_optimized_solution_and_score(y, test_delta, X_test, naive_submission, naive_scores)
print(score, naive_scores.mean() - score)

In [ ]:
submission = pd.read_csv("../input/conways-reverse-game-of-life-2020/sample_submission.csv", index_col='id')
submission_values = y_final.reshape((len(submission), 625))
for i, col in enumerate(submission.columns):
    submission[col].values[:] = submission_values[:, i]


In [ ]:
submission.to_csv("submission.csv")